In [ ]:
import pandas as pd
from tqdm.notebook import tqdm 

In [ ]:
df = pd.read_csv('weather_data.csv')
df['일시'] = pd.to_datetime(df['일시'])

# 지점, 일시 순으로 정렬
df.sort_values(by=['지점', '일시'], inplace=True, ignore_index=True)

# 누락 시간대 데이터프레임
adding_df = pd.DataFrame(columns=df.columns)

# 누락 시간대에 행 추가 - 앞뒤 평균치로 대체
for i in tqdm(range(len(df)-1)):
    diff = df['일시'][i+1] - df['일시'][i]
    
    # 앞뒤 행의 차이가 5분이 아닐 경우
    if diff.seconds != 300:

        if df['지점'][i+1] == df['지점'][i]:
            print(df['지점명'][i], df['일시'][i], diff)

            for j in range(int((diff.seconds) / 300 - 1)):
                # 새 행 추가
                new_row = {
                    '지점' : df['지점'][i],
                    '지점명': df['지점명'][i],
                    '일시' : df['일시'][i] + timedelta(minutes=5*(j+1)),
                    '기온(°C)' : (df['기온(°C)'][i] + df['기온(°C)'][i+1]) / 2,
                    '누적강수량(mm)' : (df['누적강수량(mm)'][i] + df['누적강수량(mm)'][i+1]) / 2,
                    '풍향(deg)' : (df['풍향(deg)'][i] + df['풍향(deg)'][i+1]) / 2,
                    '풍속(m/s)' : (df['풍속(m/s)'][i] + df['풍속(m/s)'][i+1]) / 2,
                    '현지기압(hPa)' : (df['현지기압(hPa)'][i] + df['현지기압(hPa)'][i+1]) / 2,
                    '해면기압(hPa)' : (df['해면기압(hPa)'][i] + df['해면기압(hPa)'][i+1]) / 2,
                    '습도(%)' : (df['습도(%)'][i] + df['습도(%)'][i+1]) / 2,
                    '일사(MJ/m^2)' : (df['일사(MJ/m^2)'][i] + df['일사(MJ/m^2)'][i+1]) / 2,
                    '일조(Sec)' : (df['일조(Sec)'][i] + df['일조(Sec)'][i+1]) / 2
                }

                adding_df = adding_df.append(new_row, ignore_index=True)
                
df = pd.concat([df,adding_df], ignore_index=True)
df['일시'] = pd.to_datetime(df['일시'])
df.sort_values(by=['지점', '일시'], inplace=True, ignore_index=True)

# csv 저장
df.to_csv('./weather_data_v3.csv', index=False)

In [ ]:
# 시간대별 데이터 만들기
df = pd.read_csv('weather_data_v3.csv')

df.sort_values(by=['일시', '지점'], inplace=True, ignore_index=True)

# 새 데이터프레임 만들기
new_cols = ['일시']
for city in df['지점명'].unique():
    for info in df.columns[3:]:
        new_cols.append(f'{city}_{info}')
        
new_df = pd.DataFrame(columns=new_cols, index=range(1135872))
new_df['일시'] = df['일시'].unique()

count = 0
# 같은 시간대 - 16,32,48,..
for i in tqdm(range(16,len(df),16)):
    # 한 지역 - 0,1,2,...
    col_set = 1
    for j in range(i-16, i):
        new_df.loc[count][col_set:col_set+9] = df.loc[j][3:]
        col_set += 9
    count += 1
    
new_df = new_df.astype('float')

# csv 저장
new_df.to_csv('./weather_data_v4.csv', index=False)

In [ ]:
new_df = pd.read_csv('./weather_data_v4.csv')

new_df.set_index('일시', drop=True, inplace=True)
new_df.fillna(new_df.interpolate(), inplace=True)

# 전력 데이터 불러오기
electric1 = pd.read_csv('6_20.csv')

electric1['기준일시'] = pd.to_datetime(electric1['기준일시'])

adding_df = pd.DataFrame(columns=electric1.columns)

# 누락 시간대에 행 추가 - 평균치로 대체
for i in tqdm(range(len(electric1)-1)):

    diff = electric1['기준일시'][i+1] - electric1['기준일시'][i]
    
    # 앞뒤 행의 차이가 5분이 아닐 경우
    if (diff.days != 0) or (diff.seconds != 300):

        print(electric1['기준일시'][i], diff)

        for j in range(int((diff.days*86400 + diff.seconds) / 300 - 1)):
            # 새 행 추가
            new_row = {
                '기준일시' : electric1['기준일시'][i] + timedelta(minutes=5*(j+1)),
                '공급능력(MW)' : (electric1['공급능력(MW)'][i] + electric1['공급능력(MW)'][i+1]) / 2,
                '현재수요(MW)' : (electric1['현재수요(MW)'][i] + electric1['현재수요(MW)'][i+1]) / 2,
                '공급예비력(MW)' : (electric1['공급예비력(MW)'][i] + electric1['공급예비력(MW)'][i+1]) / 2,
                '공급예비율(퍼센트)' : (electric1['공급예비율(퍼센트)'][i] + electric1['공급예비율(퍼센트)'][i+1]) / 2 ,               
                '운영예비력(MW)' : (electric1['운영예비력(MW)'][i] + electric1['운영예비력(MW)'][i+1]) / 2,
                '운영예비율(퍼센트)' : (electric1['운영예비율(퍼센트)'][i] + electric1['운영예비율(퍼센트)'][i+1]) / 2,
                '요일' : electric1['요일'][i],
                'is_holiday_or_weekend' : electric1['is_holiday_or_weekend'][i],
                '요일1' : electric1['요일1'][i],
                '요일2' : electric1['요일2'][i],
                '요일3' : electric1['요일3'][i],
            }
            adding_df = adding_df.append(new_row, ignore_index=True)

electric1 = pd.concat([electric1,adding_df], ignore_index=True)
electric1['기준일시'] = pd.to_datetime(electric1['기준일시'])
electric1.sort_values(by=['기준일시'], inplace=True, ignore_index=True)

# 기상 데이터 + 전력 데이터
df = new_df.join(electric1)

df.to_csv('./weather_electric_data.csv')